# Домашнее задание: Спекулятивное декодирование и архитектура Qwen

**Курс:** NLP-2

## Описание задания

В этом домашнем задании мы не будем заниматься обучением моделей с нуля. Вместо этого мы сфокусируемся на **Inference Engineering** — области, которая находится на стыке разработки и исследований и занимается оптимизацией и ускорением работы уже обученных больших языковых моделей (LLM).

Мы реализуем с нуля метод **Speculative Decoding** — один из самых популярных алгоритмических подходов, позволяющий ускорить генерацию текста в 1.5-2.5 раза практически без потери качества. Этот метод используется для оптимизации инференса таких моделей, как Llama, Mixtral и других.

### План работы:
1. **Подготовка окружения**: Настроим среду и загрузим модели.
2. **Реализация Draft модели**: Мы вручную, блок за блоком, соберем архитектуру модели `Qwen2.5-0.5B` (`NanoQwen`). Это позволит нам досконально понять устройство современных LLM, включая `RMSNorm`, `RoPE` и `SwiGLU`.
3. **Реализация цикла спекуляции**: Напишем основной алгоритм, в котором Draft модель быстро генерирует черновик, а Target модель его верифицирует.
4. **Бенчмарк**: Проведем замеры и оценим реальное ускорение, которое дает наш метод.

## Шаг 1: Настройка окружения

In [ ]:
!pip install -q transformers accelerate safetensors sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from safetensors.torch import load_file
from huggingface_hub import hf_hub_download
import time
import gc
import json
import numpy as np

# Фиксируем seed для воспроизводимости
torch.manual_seed(42)
np.random.seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


### Загрузка моделей

В качестве целевой модели (`Target`) мы будем использовать `Qwen/Qwen2.5-1.5B-Instruct`. В качестве черновой модели (`Draft`) — `Qwen/Qwen2.5-0.5B-Instruct`. Мы загрузим `Target` с помощью стандартного класса `AutoModelForCausalLM` из `transformers`, а вот `Draft` модель соберем вручную.

In [ ]:
TARGET_ID = "Qwen/Qwen2.5-1.5B-Instruct"
DRAFT_ID = "Qwen/Qwen2.5-0.5B-Instruct"

print("Загрузка токенизатора...")
tokenizer = AutoTokenizer.from_pretrained(TARGET_ID)

print("Загрузка Target модели (через AutoModelForCausalLM)...")
# Используем torch_dtype=torch.float16 для экономии VRAM и attn_implementation="sdpa" для использования Flash Attention
target_model = AutoModelForCausalLM.from_pretrained(
    TARGET_ID,
    torch_dtype=torch.float16,
    device_map='cuda',
    attn_implementation="sdpa"
)
target_model.eval()
print("Модель Target загружена.")

Загрузка токенизатора...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Модель Target загружена.


## Шаг 2: Собираем Draft модель "NanoQwen" (4 балла)

На этом шаге мы не будем использовать `AutoModel`, а соберем модель самостоятельно из отдельных модулей. Это ключевая часть задания, которая поможет понять, как современные трансформеры устроены "под капотом".

Сначала загрузим только конфигурацию `Draft` модели, из которой мы будем брать все параметры архитектуры (размер скрытого слоя, количество голов и т.д.).

In [ ]:
draft_config = AutoConfig.from_pretrained(DRAFT_ID)
print(f"Конфигурация Draft модели:")
print(f"  - Размер скрытого слоя (hidden_size): {draft_config.hidden_size}")
print(f"  - Количество слоев (num_hidden_layers): {draft_config.num_hidden_layers}")
print(f"  - Количество голов внимания (num_attention_heads): {draft_config.num_attention_heads}")

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

Конфигурация Draft модели:
  - Размер скрытого слоя (hidden_size): 896
  - Количество слоев (num_hidden_layers): 24
  - Количество голов внимания (num_attention_heads): 14


### Задание 2.1: RMSNorm

Современные модели, такие как Llama и Qwen, используют **Root Mean Square Layer Normalization (RMSNorm)** вместо классического `LayerNorm`. `RMSNorm` проще и вычислительно эффективнее, так как оперирует только масштабированием на основе среднеквадратичного значения и не использует дополнительный сдвиг (bias).

Формула `RMSNorm` для вектора активаций $\mathbf{x}$:
$$ \text{RMS}(\mathbf{x}) = \sqrt{\frac{1}{n} \sum_{i=1}^{n} x_i^2} $$
$$ \text{RMSNorm}(\mathbf{x}) = \frac{\mathbf{x}}{\text{RMS}(\mathbf{x}) + \epsilon} \cdot \mathbf{w} $$
где $\mathbf{w}$ — обучаемый весовой вектор (гейт), а $\epsilon$ — малая константа для численной стабильности.

**Важно**: При вычислениях в `float16`, промежуточный расчет `variance` (степень `pow(2)`) может привести к переполнению. Поэтому стандартная практика — временно повышать тип данных до `float32` для этого расчета.

**Задание**: Реализуйте `forward` для `QwenRMSNorm`.

**Полезные ссылки**:
- [Root Mean Square Layer Normalization (Zhang and Sennrich, 2019)](https://arxiv.org/abs/1910.07467)

In [ ]:
class QwenRMSNorm(nn.Module):
    """
    Реализация Root Mean Square Layer Normalization.

    Аргументы:
        hidden_size (int): Размер скрытого слоя.
        eps (float): Малая константа для предотвращения деления на ноль.
    """
    def __init__(self, hidden_size, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        # --- НАЧАЛО ВАШЕГО КОДА ---
        pass
        # 1. Запомните исходный тип данных (например, float16).
        input_dtype = hidden_states.dtype

        # 2. Переведите hidden_states в float32 для стабильных вычислений.
        hidden_states = hidden_states.to(torch.float32)

        # 3. Вычислите variance: среднее от квадратов элементов по последней оси.
        #    Не забудьте `keepdim=True`.
        variance = hidden_states.pow(2).mean(-1, keepdim=True)

        # 4. Нормализуйте hidden_states (torch.rsqrt в помощь)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)

        # 5. Умножьте на обучаемый вес и верните результат в исходном типе данных.
        return (hidden_states * self.weight).to(input_dtype)
        # --- КОНЕЦ ВАШЕГО КОДА ---


In [ ]:
print("--- Запуск тестов для RMSNorm ---")
try:
    hidden_size = 128
    norm_layer = QwenRMSNorm(hidden_size).to(device).half()

    # Тест 1: Проверка размерности
    dummy_input = torch.randn(2, 10, hidden_size, device=device).half()
    output = norm_layer(dummy_input)
    assert output.shape == dummy_input.shape, f"Ошибка размерности: ожидалось {dummy_input.shape}, получено {output.shape}"
    print("✅ [1/3] Тест на размерность пройден.")

    # Тест 2: Проверка типа данных
    assert output.dtype == torch.float16, f"Ошибка типа данных: ожидалось float16, получено {output.dtype}"
    print("✅ [2/3] Тест на тип данных пройден.")

    # Тест 3: Проверка на NaN
    assert not torch.isnan(output).any(), "В выходе RMSNorm обнаружены NaN значения."
    print("✅ [3/3] Тест на NaN пройден.")

    print("\n🎉 Все тесты для RMSNorm пройдены!")

except Exception as e:
    print(f"❌ Тест RMSNorm провален: {e}")

--- Запуск тестов для RMSNorm ---
✅ [1/3] Тест на размерность пройден.
✅ [2/3] Тест на тип данных пройден.
✅ [3/3] Тест на NaN пройден.

🎉 Все тесты для RMSNorm пройдены!


### Задание 2.2: Rotary Positional Embeddings (RoPE)

`RoPE` — это элегантный способ внедрения позиционной информации, который вместо добавления векторов (как в `sin/cos embeddings`) "вращает" векторы запросов (`Query`) и ключей (`Key`) на угол, зависящий от их позиции.

#### Два подхода к реализации
Существует два эквивалентных способа реализации этого вращения.

1. **Разбиение на пары (Pairwise Rotation)**. Этот метод мы рассматривали на лекции. Вектор признаков $\mathbf{x} = (x_1, x_2, \dots, x_d)$ рассматривается как набор двумерных векторов $(x_{2i-1}, x_{2i})$. Каждый такой вектор вращается в 2D-плоскости:
$$
\begin{pmatrix} x'_{2i-1} \\ x'_{2i} \end{pmatrix} =
\begin{pmatrix} \cos(m\theta_i) & -\sin(m\theta_i) \\ \sin(m\theta_i) & \cos(m\theta_i) \end{pmatrix}
\begin{pmatrix} x_{2i-1} \\ x_{2i} \end{pmatrix}
$$
где $m$ — позиция токена, а $\theta_i$ — частота вращения.

2. **Вращение через половины (`rotate_half`)**. Этот метод используется в реализациях Llama и Qwen, и именно его мы будем использовать. Здесь вектор $\mathbf{x}$ делится на две половины: $\mathbf{x}_1 = (x_1, \dots, x_{d/2})$ и $\mathbf{x}_2 = (x_{d/2+1}, \dots, x_d)$. Вращение реализуется следующим образом:
$$ \mathbf{x}'_m = \mathbf{x}_m \cos(m\theta) + \text{rotate\_half}(\mathbf{x}_m) \sin(m\theta) $$
где операция `rotate_half` преобразует вектор $\mathbf{x} = (\mathbf{x}_1, \mathbf{x}_2)$ в $(-\mathbf{x}_2, \mathbf{x}_1)$. Этот подход легче векторизуется и более эффективен в `torch`.

**Задание**: Реализуйте функцию `apply_rope`, следуя второму подходу.

**Полезные ссылки**:
- [RoFormer: Enhanced Transformer with Rotary Position Embedding (Su et al., 2021)](https://arxiv.org/abs/2104.09864)
- [Подробное объяснение RoPE в блоге EleutherAI](https://blog.eleuther.ai/rotary-embeddings/)

In [ ]:
def precompute_freqs_cis(dim: int, end: int, theta: float = 1000000.0):
    """
    Предварительно вычисляет частоты для RoPE в комплексном виде (cos + i*sin).

    Эта функция готовит таблицы синусов и косинусов для всех возможных позиций.
    """
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(end, device=freqs.device, dtype=torch.float32)
    freqs = torch.outer(t, freqs)
    # В реализации Llama/Qwen частоты дублируются для обеих половин
    # Мы вернем косинусы и синусы отдельно для удобства
    freqs_cat = torch.cat((freqs, freqs), dim=-1)
    return torch.cos(freqs_cat), torch.sin(freqs_cat)

def rotate_half(x):
    """Вращает половину скрытых измерений входа."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)

def apply_rope(q, k, cos, sin):
    """
    Применяет RoPE к векторам q и k.
    """
    # --- НАЧАЛО ВАШЕГО КОДА ---
    # 1. Измените размерность cos и sin для бродкастинга с (q, k).
    #    Они должны стать [1, seq_len, 1, head_dim].
    cos = cos.unsqueeze(0).unsqueeze(2)
    sin = sin.unsqueeze(0).unsqueeze(2)

    # 2. Примените формулу вращения к q и k, используя rotate_half.
    #    Операции с float32 (cos/sin) приведут к результату float32.
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)

    # 3. Верните q_embed и k_embed.
    #    ВАЖНО: Явно приводим к типу q.dtype (float16), иначе упадет assert или следующий слой
    return q_embed.to(q.dtype), k_embed.to(k.dtype)

    # --- КОНЕЦ ВАШЕГО КОДА ---


In [ ]:
print("--- Запуск тестов для RoPE ---")
try:
    head_dim = 64
    seq_len = 10
    batch_size = 2
    num_heads = 4

    xq = torch.randn(batch_size, seq_len, num_heads, head_dim, device=device).half()
    xk = torch.randn(batch_size, seq_len, num_heads, head_dim, device=device).half()
    cos, sin = precompute_freqs_cis(head_dim, seq_len * 2)
    cos, sin = cos.to(device), sin.to(device)

    # Вызываем функцию
    xq_rot, xk_rot = apply_rope(xq, xk, cos[:seq_len], sin[:seq_len])

    # Тест 1: Проверка размерности
    assert xq_rot.shape == xq.shape, f"Ошибка размерности Query: ожидалось {xq.shape}, получено {xq_rot.shape}"
    assert xk_rot.shape == xk.shape, f"Ошибка размерности Key: ожидалось {xk.shape}, получено {xk_rot.shape}"
    print("✅ [1/2] Тест на размерность пройден.")

    # Тест 2: Проверка типа данных
    assert xq_rot.dtype == xq.dtype, f"Ошибка типа данных Query: ожидалось {xq.dtype}, получено {xq_rot.dtype}"
    assert xk_rot.dtype == xk.dtype, f"Ошибка типа данных Key: ожидалось {xk.dtype}, получено {xk_rot.dtype}"
    print("✅ [2/2] Тест на тип данных пройден.")

    print("\n🎉 Все тесты для RoPE пройдены!")
except Exception as e:
    print(f"❌ Тест RoPE провален: {e}")

--- Запуск тестов для RoPE ---
✅ [1/2] Тест на размерность пройден.
✅ [2/2] Тест на тип данных пройден.

🎉 Все тесты для RoPE пройдены!


### Задание 2.3: SwiGLU MLP

Вместо стандартного `FeedForward` блока с одной `ReLU` активацией, современные модели используют `Gated Linear Units (GLU)` и их варианты. В Qwen/Llama используется **SwiGLU**.

Идея состоит в том, чтобы использовать гейт (шлюз) для управления информационным потоком. Входной вектор `x` проецируется двумя разными линейными слоями (`up` и `gate`). Результат `gate` проекции проходит через активацию `SiLU` (также известную как Swish), а затем поэлементно умножается на результат `up` проекции. Это позволяет сети динамически решать, какая информация должна пройти дальше.

Формула `SwiGLU`:
$$ \text{SwiGLU}(x, W_{up}, W_{gate}, W_{down}) = (\text{SiLU}(x W_{gate}) \otimes (x W_{up})) W_{down} $$
где $\otimes$ — поэлементное умножение, а `SiLU` (Swish activation) определяется как:
$$ \text{SiLU}(x) = x \cdot \sigma(x) $$
где $\sigma$ — это сигмоида.

**Задание**: Реализуйте `forward` для `QwenMLP`.

**Полезные ссылки**:
- [GLU Variants Improve Transformer (Shazeer, 2020)](https://arxiv.org/abs/2002.05202)

In [ ]:
class QwenMLP(nn.Module):
    """
    Реализация SwiGLU Feed-Forward сети.
    """
    def __init__(self, config):
        super().__init__()
        self.gate_proj = nn.Linear(config.hidden_size, config.intermediate_size, bias=False)
        self.up_proj = nn.Linear(config.hidden_size, config.intermediate_size, bias=False)
        self.down_proj = nn.Linear(config.intermediate_size, config.hidden_size, bias=False)

    def forward(self, x):
        # --- НАЧАЛО ВАШЕГО КОДА ---
        pass
        # 1. Примените gate_proj и up_proj к входу x.
        gate = self.gate_proj(x)
        up = self.up_proj(x)

        # 2. Примените активацию SiLU (F.silu) к выходу gate_proj.
        gate = F.silu(gate)

        # 3. Поэлементно перемножьте результат шага 2 и выход up_proj.
        combined = gate * up

        # 4. Пропустите результат через down_proj и верните его.
        return self.down_proj(combined)

        # --- КОНЕЦ ВАШЕГО КОДА ---


In [ ]:
print("--- Запуск тестов для SwiGLU MLP ---")
try:
    # Создаем mock-конфиг для теста
    class MockConfig:
        hidden_size = 128
        intermediate_size = 384

    config = MockConfig()
    mlp = QwenMLP(config).to(device).half()

    x = torch.randn(2, 10, config.hidden_size, device=device).half()
    output = mlp(x)

    # Тест 1: Проверка размерности
    assert output.shape == x.shape, f"Ошибка размерности: ожидалось {x.shape}, получено {output.shape}"
    print("✅ [1/2] Тест на размерность пройден.")

    # Тест 2: Проверка на NaN
    assert not torch.isnan(output).any(), "В выходе MLP обнаружены NaN значения."
    print("✅ [2/2] Тест на NaN пройден.")

    print("\n🎉 Все тесты для SwiGLU MLP пройдены!")
except Exception as e:
    print(f"❌ Тест SwiGLU MLP провален: {e}")

--- Запуск тестов для SwiGLU MLP ---
✅ [1/2] Тест на размерность пройден.
✅ [2/2] Тест на NaN пройден.

🎉 Все тесты для SwiGLU MLP пройдены!


### Задание 2.4: Сборка итоговой модели NanoQwen

Теперь, когда у нас есть все строительные блоки, мы можем собрать из них полноценный слой трансформера (`NanoQwenBlock`) и саму модель (`NanoQwen`).

Вам необходимо дополнить класс `NanoQwenBlock`, правильно соединив все модули. Обратите внимание на порядок операций в трансформерах семейства Llama/Qwen:
1. **Pre-normalization** перед Self-Attention.
2. **Residual Connection** после Self-Attention.
3. **Pre-normalization** перед MLP.
4. **Residual Connection** после MLP.

In [ ]:
class QwenAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.hidden_size = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = config.hidden_size // config.num_attention_heads
        self.num_key_value_heads = config.num_key_value_heads

        self.q_proj = nn.Linear(self.hidden_size, self.num_heads * self.head_dim, bias=True)
        self.k_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=True)
        self.v_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=True)
        self.o_proj = nn.Linear(self.num_heads * self.head_dim, self.hidden_size, bias=False)

    def forward(self, hidden_states, cos, sin):
        bsz, q_len, _ = hidden_states.size()

        q = self.q_proj(hidden_states).view(bsz, q_len, self.num_heads, self.head_dim)
        k = self.k_proj(hidden_states).view(bsz, q_len, self.num_key_value_heads, self.head_dim)
        v = self.v_proj(hidden_states).view(bsz, q_len, self.num_key_value_heads, self.head_dim)

        q, k = apply_rope(q, k, cos, sin)

        # Grouped Query Attention (GQA)
        if self.num_key_value_heads != self.num_heads:
            k = k.repeat_interleave(self.num_heads // self.num_key_value_heads, dim=2)
            v = v.repeat_interleave(self.num_heads // self.num_key_value_heads, dim=2)

        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        # Используем встроенную реализацию Flash Attention
        output = F.scaled_dot_product_attention(q, k, v, is_causal=True)

        output = output.transpose(1, 2).contiguous().view(bsz, q_len, -1)
        return self.o_proj(output)

class NanoQwenBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.self_attn = QwenAttention(config)
        self.mlp = QwenMLP(config)
        self.input_layernorm = QwenRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.post_attention_layernorm = QwenRMSNorm(config.hidden_size, eps=config.rms_norm_eps)

    def forward(self, hidden_states, cos, sin):
        # --- НАЧАЛО ВАШЕГО КОДА ---

        # 1. Pre-normalization и Self-Attention
        residual = hidden_states
        hidden_states = self.input_layernorm(hidden_states)
        hidden_states = self.self_attn(hidden_states, cos, sin)
        hidden_states = residual + hidden_states

        # 2. Pre-normalization и MLP
        residual = hidden_states
        hidden_states = self.post_attention_layernorm(hidden_states)
        hidden_states = self.mlp(hidden_states)
        hidden_states = residual + hidden_states

        # --- КОНЕЦ ВАШЕГО КОДА ---
        return hidden_states

class NanoQwen(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size)
        self.layers = nn.ModuleList([NanoQwenBlock(config) for _ in range(config.num_hidden_layers)])
        self.norm = QwenRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

        # Предвычисляем RoPE
        self.cos, self.sin = precompute_freqs_cis(
            config.hidden_size // config.num_attention_heads,
            config.max_position_embeddings * 2
        )
        self.cos = self.cos.to(device)
        self.sin = self.sin.to(device)

    def forward(self, input_ids):
        x = self.embed_tokens(input_ids)
        seq_len = x.shape[1]

        # Выбираем нужный срез из таблицы RoPE
        cos_t = self.cos[:seq_len]
        sin_t = self.sin[:seq_len]

        for layer in self.layers:
            x = layer(x, cos_t, sin_t)

        x = self.norm(x)
        logits = self.lm_head(x)

        return logits

### Загрузка весов

Теперь нам нужно загрузить веса из официального репозитория в нашу самописную модель. Мы уже реализовали функцию `load_weights_manual`, которая делает это и корректно обрабатывает различия в именовании слоев.

In [ ]:
def load_weights_manual(model, model_id):
    """
    Загружает веса из файла .safetensors в ручном режиме,
    корректируя имена ключей.
    """
    print("Скачивание весов...")
    file_path = hf_hub_download(repo_id=model_id, filename="model.safetensors")

    print("Загрузка файла safetensors...")
    state_dict = load_file(file_path, device="cpu") # Грузим на CPU, чтобы не занимать VRAM

    new_state_dict = {}
    for key, tensor in state_dict.items():
        new_key = key

        # Если ключ начинается с "model.", удаляем этот префикс
        if key.startswith("model."):
            new_key = key[len("model."):]

        new_state_dict[new_key] = tensor

    # В некоторых моделях lm_head и embed_tokens используют общие веса (tied weights).
    # Если lm_head нет, копируем его из embed_tokens.
    if "lm_head.weight" not in new_state_dict and "embed_tokens.weight" in new_state_dict:
        print("Связывание весов: lm_head.weight <- embed_tokens.weight")
        new_state_dict["lm_head.weight"] = new_state_dict["embed_tokens.weight"]

    print("Загрузка весов в модель...")
    missing, unexpected = model.load_state_dict(new_state_dict, strict=False)

    if not missing and not unexpected:
        print("✅ Веса успешно загружены!")
    else:
        print(f"❌ Ошибка при загрузке весов:")
        if missing:
            print(f"  Не найдены ключи в state_dict: {missing[:5]}")
        if unexpected:
            print(f"  Лишние ключи в state_dict: {unexpected[:5]}")

    model.to(device).to(torch.float16)
    return model, missing, unexpected

In [ ]:
# Инициализация и загрузка Draft модели
draft_model = NanoQwen(draft_config)
draft_model, missing_keys, unexpected_keys = load_weights_manual(draft_model, DRAFT_ID)
draft_model.eval()

# Тест
print("\n--- Запуск теста для загрузки весов ---")
try:
    assert not missing_keys, f"Найдены недостающие ключи: {missing_keys[:5]}"
    assert not unexpected_keys, f"Найдены лишние ключи: {unexpected_keys[:5]}"
    print("🎉 Тест на загрузку весов пройден!")
except Exception as e:
    print(f"❌ Тест провален: {e}")

Скачивание весов...


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Загрузка файла safetensors...
Связывание весов: lm_head.weight <- embed_tokens.weight
Загрузка весов в модель...
✅ Веса успешно загружены!

--- Запуск теста для загрузки весов ---
🎉 Тест на загрузку весов пройден!


### Проверка работоспособности

Давайте убедимся, что наша самописная модель генерирует осмысленный текст. Мы используем простой цикл жадной генерации (`greedy search`).

In [ ]:
def generate_simple(model, text, max_new=10):
    """Простая функция для жадной генерации."""
    inputs = tokenizer(text, return_tensors="pt").to(device)
    input_ids = inputs.input_ids

    with torch.no_grad(), torch.amp.autocast(device_type="cuda", dtype=torch.float16):
        for _ in range(max_new):
            logits = model(input_ids)
            next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
            input_ids = torch.cat([input_ids, next_token], dim=1)
            if next_token.item() == tokenizer.eos_token_id:
                break

    return tokenizer.decode(input_ids[0], skip_special_tokens=True)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who are you?"}
]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print("--- Ответ от NanoQwen: ---")
print(generate_simple(draft_model, text, max_new=20))

--- Ответ от NanoQwen: ---
system
You are a helpful assistant.
user
Who are you?
assistant
I am a large language model created by Alibaba Cloud. I am called Qwen.


## Шаг 3: Алгоритм спекулятивного декодирования (3 балла)

Теперь самая важная часть. Мы реализуем **Greedy Speculative Decoding**.

**Алгоритм**:
1. **Черновик (Draft)**: Генерируем `K` токенов с помощью быстрой маленькой модели (`Draft`).
2. **Верификация (Verify)**: Прогоняем всю последовательность (префикс + `K` токенов) через медленную большую модель (`Target`) **за один `forward` вызов**.
3. **Проверка (Accept/Reject)**: Сравниваем токены, предсказанные `Target` моделью, с токенами, сгенерированными `Draft` моделью.
   - Находим первый индекс `i`, где предсказания не совпали.
   - Все `i` совпавших токенов считаются "принятыми" и добавляются к результату.
   - В качестве следующего токена мы берем "правильный" токен от `Target` модели на позиции `i`.
   - Все остальные токены из черновика отбрасываются.
4. Повторяем цикл.

**Задание**: Реализуйте логику проверки и принятия токенов.

**Полезные ссылки**:
- [Fast Inference from Transformers via Speculative Decoding (Leviathan et al., 2022)](https://arxiv.org/abs/2211.17192)

In [ ]:
def speculative_sampling(prefix_text, max_new_tokens, target_model, draft_model, tokenizer, K=5):
    """
    Реализует цикл спекулятивного декодирования.
    """
    # Форматируем промпт
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prefix_text}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)

    generated_ids = input_ids.clone()
    finished_len = input_ids.shape[1] + max_new_tokens

    stats = {"target_calls": 0, "total_accepted": 0, "total_drafted": 0}

    with torch.no_grad(), torch.amp.autocast(device_type="cuda", dtype=torch.float16):
        while generated_ids.shape[1] < finished_len:
            prefix_len = generated_ids.shape[1]

            # 1. DRAFT
            draft_ids = generated_ids
            for _ in range(K):
                outputs = draft_model(draft_ids)
                next_token = torch.argmax(outputs[:, -1, :], dim=-1, keepdim=True)
                draft_ids = torch.cat([draft_ids, next_token], dim=1)
                if next_token.item() == tokenizer.eos_token_id:
                    break

            drafted_tokens = draft_ids[0, prefix_len:]
            if not len(drafted_tokens): break # Если ничего не сгенерировали
            stats["total_drafted"] += len(drafted_tokens)

            # 2. VERIFY
            target_outputs = target_model(draft_ids)
            stats["target_calls"] += 1
            target_logits = target_outputs.logits
            # Нас интересуют предсказания Target модели для токенов, которые сгенерировал Draft
            relevant_logits = target_logits[0, prefix_len-1 : -1]
            target_preds = torch.argmax(relevant_logits, dim=-1)

            # 3. ACCEPT/REJECT Logic
            n_accepted = 0

            # --- НАЧАЛО ВАШЕГО КОДА ---
            # Проитерируйтесь по `drafted_tokens` и `target_preds`.
                # Увеличивайте `n_accepted`, пока токены совпадают.
                    # Прервите цикл, как только найдете первое несовпадение.
            for i in range(len(drafted_tokens)):
                if i >= len(target_preds):
                    break
                if drafted_tokens[i] == target_preds[i]:
                    n_accepted += 1
                else:
                    break
            # --- КОНЕЦ ВАШЕГО КОДА ---

            stats["total_accepted"] += n_accepted

            # Принимаем все совпавшие токены
            accepted_ids = drafted_tokens[:n_accepted]
            generated_ids = torch.cat([generated_ids, accepted_ids.unsqueeze(0)], dim=1)

            # Если достигли лимита, выходим
            if generated_ids.shape[1] >= finished_len:
                break

            # Добавляем один "исправленный" токен от Target модели
            if n_accepted < len(target_preds):
                correct_token = target_preds[n_accepted].view(1, 1)
            else: # Если все совпало, берем следующий токен от Target модели
                last_logits = target_logits[0, -1, :]
                correct_token = torch.argmax(last_logits).view(1, 1)

            generated_ids = torch.cat([generated_ids, correct_token], dim=1)

            if correct_token.item() == tokenizer.eos_token_id:
                break

    return tokenizer.decode(generated_ids[0], skip_special_tokens=True), stats

In [ ]:
print("--- Запуск теста для Speculative Decoding ---")
try:
    # Создаем "игрушечные" модели для теста
    class MockModel(nn.Module):
        def __init__(self, vocab_size, response_sequence):
            super().__init__()
            self.vocab_size = vocab_size
            self.response = response_sequence
            self.call_idx = 0
        def forward(self, input_ids):
            if input_ids is None: return None # property hack fix
            batch, seq_len = input_ids.shape
            next_token_idx = min(self.call_idx, len(self.response) -1)
            next_token = self.response[next_token_idx]
            self.call_idx += 1
            logits = torch.full((batch, seq_len, self.vocab_size), -100.0, device=device)
            logits[:, -1, next_token] = 100.0
            return logits

    mock_draft = MockModel(100, [10, 20, 30, 40, 50])
    mock_target = MockModel(100, [10, 20, 99, 40, 50])
    # Хамский хак для property, но в тесте мы вызываем forward напрямую через target(draft_ids)

    def mock_speculative_sampling(target, draft, K=5):
        generated_ids = torch.tensor([[1, 2]], device=device)
        prefix_len = generated_ids.shape[1]
        draft_ids = torch.tensor([[1, 2, 10, 20, 30, 40, 50]], device=device)

        drafted_tokens = draft_ids[0, prefix_len:]
        print(f"Drafted tokens: {drafted_tokens.tolist()}")

        # Эмулируем вызов target модели
        # Важно: MockModel в нашем тесте очень тупая, она просто возвращает следующий токен из списка response
        # независимо от входа. Нам нужно вызвать ее для каждого токена последовательности,
        # чтобы собрать "правильные" предикты.

        # Перепишем логику сбора предиктов для MockModel, чтобы она работала как авторегрессия
        target_preds_list = []

        # Мы хотим проверить предикты для:
        # input=[1, 2] -> предсказание (должно быть 10)
        # input=[... 10] -> предсказание (должно быть 20)
        # input=[... 20] -> предсказание (должно быть 99)

        # Сбросим состояние
        target.call_idx = 0

        # В реальной жизни мы делаем один forward pass.
        # Но наша MockModel возвращает только последний токен.
        # Поэтому для теста мы просто возьмем response_sequence учителя.
        # Это упрощение теста, но оно валидирует логику accept/reject.

        teacher_sequence = target.response # [10, 20, 99, 40, 50]
        target_preds = torch.tensor(teacher_sequence, device=device)

        print(f"Target preds (ideal): {target_preds.tolist()}")

        n_accepted = 0
        # --- КОПИЯ ВАШЕЙ ЛОГИКИ ИЗ speculative_sampling ---
        # raise NotImplementedError("КОПИЯ ВАШЕЙ ЛОГИКИ ИЗ speculative_sampling")
        for i in range(len(drafted_tokens)):
            if i >= len(target_preds):
                break
            if drafted_tokens[i] == target_preds[i]:
                n_accepted += 1
            else:
                break
        # --------------------------------------------------
        return n_accepted

    n_accepted = mock_speculative_sampling(mock_target, mock_draft)
    print(f"Accepted: {n_accepted}")

    assert n_accepted == 2, f"Ошибка в логике Accept/Reject: ожидалось 2 принятых токена, получено {n_accepted}"
    print("🎉 Тест для логики Accept/Reject пройден!")

except Exception as e:
    print(f"❌ Тест провален: {e}")


--- Запуск теста для Speculative Decoding ---
Drafted tokens: [10, 20, 30, 40, 50]
Target preds (ideal): [10, 20, 99, 40, 50]
Accepted: 2
🎉 Тест для логики Accept/Reject пройден!


## Шаг 4: Бенчмарк

Чтобы увидеть реальный выигрыш от спекулятивного декодирования, нам нужно симулировать ситуацию, когда `Target` модель работает значительно медленнее, чем `Draft`. В реальной жизни так и происходит: `Draft` может быть модель на 0.5B параметров, а `Target` — на 70B.

Мы создадим обертку `HeavyTarget`, которая будет искусственно добавлять задержку перед каждым вызовом `forward`, имитируя медленный инференс большой модели. Затем мы сравним время генерации стандартным (авторегрессионным) способом и с помощью нашего спекулятивного алгоритма.Ожидается, что спекулятивное декодирование покажет значительное ускорение (speedup ~ 1.5x+).

Однако, обратите внимание, что для действительно эффективной работы на длинных контекстах нам необходим KV-cache, чтобы заново не пересчитывать уже выполненные вычисления. Именно так speculative decoding реализован в популярных фреймворках.

In [ ]:
import pandas as pd

class HeavyTarget:
    def __init__(self, model, delay=0.05):
        self.model = model
        self.delay = delay
    def __call__(self, *args, **kwargs):
        time.sleep(self.delay)
        return self.model(*args, **kwargs)
    @property
    def config(self): return self.model.config

def autoregressive(model, text, max_new=50):
    ids = tokenizer(text, return_tensors="pt").to(device).input_ids
    start = time.time()
    cnt = 0
    with torch.no_grad():
        for _ in range(max_new):
            out = model(ids)
            tok = torch.argmax(out.logits[:, -1, :], dim=-1, keepdim=True)
            ids = torch.cat([ids, tok], dim=1)
            cnt += 1
            if tok.item() == tokenizer.eos_token_id: break
    return cnt, time.time() - start

prompts = [
    "Write a Python function to calculate Fibonacci numbers.",
    "The capital of France is",
    "Explain the theory of relativity in simple terms."
]
heavy_target = HeavyTarget(target_model, 0.04)

results = []
print("Running Benchmark...")
for p in prompts:
    # Std
    s_tok, s_time = autoregressive(heavy_target, p)
    s_speed = s_tok / s_time

    # Spec
    start = time.time()
    _, stats = speculative_sampling(p, 50, heavy_target, draft_model, tokenizer)
    spec_time = time.time() - start
    spec_tok = stats['total_accepted'] + stats['target_calls']
    spec_speed = spec_tok / spec_time

    results.append({
        "Prompt": p[:20],
        "Std (t/s)": s_speed,
        "Spec (t/s)": spec_speed,
        "Speedup": spec_speed / s_speed
    })

print(pd.DataFrame(results).to_string(float_format="{:.2f}".format))

Running Benchmark...
                 Prompt  Std (t/s)  Spec (t/s)  Speedup
0  Write a Python funct      11.23       27.61     2.46
1  The capital of Franc      11.31       16.11     1.42
2  Explain the theory o      11.95       16.12     1.35


## Что дальше?

Чтобы получить максимальный балл за ДЗ, попробуйте реализовать одну из следующих идей:

### 1. Quantized Draft Model (0.5 балла)
Мы ускорили модель алгоритмически. А давайте теперь ускорим Draft модель аппаратно! Попробуйте квантовать `NanoQwen` в 4-бит (используя библиотеки `bitsandbytes` или `GPTQ`) и посмотрите, как изменится время генерации драфтов и итоговое ускорение (Speedup).

### 2. LoRA Distillation (1 балл)
Представьте, что Draft модель плохо согласована с Target моделью. Попробуйте дообучить (Fine-Tune) Draft модель на наборе ответов Target модели, используя LoRA (Low-Rank Adaptation). Даже 500 примеров и 10-15 минут обучения на T4 могут повысить Acceptance Rate.

### 3. Layer Pruning Distillation (1.5 балла)
Возьмите Target модель (1.5B) и создайте из нее Draft модель, просто удалив половину слоев (например, каждый второй). Получится "покалеченная" модель ~0.8B. Затем проведите Knowledge Distillation (обучите её восстанавливать логиты оригинальной модели) и используйте результат как Draft модель.

### 1. Quantized Draft Model

In [ ]:
!pip install -q bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.9 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BitsAndBytesConfig
from bitsandbytes.nn import Linear4bit
import time
import pandas as pd

In [ ]:
class QuantizedNanoQwen(nn.Module):
    def __init__(self, original_model):
        super().__init__()
        self.config = original_model.config
        self.embed_tokens = original_model.embed_tokens

        # квантуем все линейные слои в блоках
        self.layers = nn.ModuleList()
        for layer in original_model.layers:
            quant_block = NanoQwenBlock(self.config)

            # квантуем q_proj, k_proj, v_proj, o_proj в attention
            quant_block.self_attn.q_proj = self._quantize_linear(layer.self_attn.q_proj)
            quant_block.self_attn.k_proj = self._quantize_linear(layer.self_attn.k_proj)
            quant_block.self_attn.v_proj = self._quantize_linear(layer.self_attn.v_proj)
            quant_block.self_attn.o_proj = self._quantize_linear(layer.self_attn.o_proj)

            # квантуем MLP проекции
            quant_block.mlp.gate_proj = self._quantize_linear(layer.mlp.gate_proj)
            quant_block.mlp.up_proj = self._quantize_linear(layer.mlp.up_proj)
            quant_block.mlp.down_proj = self._quantize_linear(layer.mlp.down_proj)

            # копируем нормы
            quant_block.input_layernorm = layer.input_layernorm
            quant_block.post_attention_layernorm = layer.post_attention_layernorm
            self.layers.append(quant_block)

        self.norm = original_model.norm
        self.lm_head = self._quantize_linear(original_model.lm_head)
        self.cos = original_model.cos
        self.sin = original_model.sin

    def _quantize_linear(self, linear_layer):
        """Квантуем линейный слой в 4-бита"""
        if not isinstance(linear_layer, nn.Linear):
            return linear_layer

        quant_layer = Linear4bit(
            linear_layer.in_features,
            linear_layer.out_features,
            bias=hasattr(linear_layer, 'bias') and linear_layer.bias is not None,
            compute_dtype=torch.float16,
            compress_statistics=True
        )
        # скопируем веса и загрузим в квантованный слой
        quant_layer.weight.data = linear_layer.weight.data
        if hasattr(linear_layer, 'bias') and linear_layer.bias is not None:
            quant_layer.bias.data = linear_layer.bias.data
        return quant_layer

    def forward(self, input_ids):
        x = self.embed_tokens(input_ids)
        seq_len = x.shape[1]
        cos_t = self.cos[:seq_len]
        sin_t = self.sin[:seq_len]

        for layer in self.layers:
            x = layer(x, cos_t, sin_t)

        x = self.norm(x)
        logits = self.lm_head(x)
        return logits

In [ ]:
# cоздаём 4-битную квантованную версию Draft модели
quantized_draft_model = QuantizedNanoQwen(draft_model).to(device)
quantized_draft_model.eval()

QuantizedNanoQwen(
  (embed_tokens): Embedding(151936, 896)
  (layers): ModuleList(
    (0-23): 24 x NanoQwenBlock(
      (self_attn): QwenAttention(
        (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
        (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
        (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
        (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
      )
      (mlp): QwenMLP(
        (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
        (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
        (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
      )
      (input_layernorm): QwenRMSNorm()
      (post_attention_layernorm): QwenRMSNorm()
    )
  )
  (norm): QwenRMSNorm()
  (lm_head): Linear4bit(in_features=896, out_features=151936, bias=False)
)

In [ ]:
def benchmark_quantized_model():
    heavy_target = HeavyTarget(target_model, 0.05)
    prompts = [
        "Write a Python function to calculate Fibonacci numbers.",
        "The capital of France is",
        "Explain the theory of relativity in simple terms."
    ]

    results = []
    print("Запуск бенчмарка для квантованной модели...")

    for p in prompts:
        # автогрессивное декодирование
        s_tok, s_time = autoregressive(heavy_target, p)
        s_speed = s_tok / s_time

        # спекулятивное декодирование с квантованной моделью Draft
        start = time.time()
        _, stats = speculative_sampling(p, 50, heavy_target, quantized_draft_model, tokenizer)
        spec_time = time.time() - start
        spec_tok = stats['total_accepted'] + stats['target_calls']
        spec_speed = spec_tok / spec_time

        results.append({
            "Prompt": p[:20],
            "Std (t/s)": s_speed,
            "Quantized Spec (t/s)": spec_speed,
            "Speedup vs Std": spec_speed / s_speed
        })

    return pd.DataFrame(results)

In [ ]:
quant_results = benchmark_quantized_model()
print("\nРезультаты бенчмарка для квантованной модели:")
print(quant_results.to_string(float_format="{:.2f}".format))

Запуск бенчмарка для квантованной модели...

Результаты бенчмарка для квантованной модели:
                 Prompt  Std (t/s)  Quantized Spec (t/s)  Speedup vs Std
0  Write a Python funct      10.70                  9.04            0.84
1  The capital of Franc      10.43                  2.26            0.22
2  Explain the theory o      10.74                  8.72            0.81


In [ ]:
def compare_models():
    heavy_target = HeavyTarget(target_model, 0.05)
    prompt = "Write a Python function to calculate Fibonacci numbers."

    # оригинальная Draft модель
    start = time.time()
    _, orig_stats = speculative_sampling(prompt, 50, heavy_target, draft_model, tokenizer)
    orig_time = time.time() - start

    # квантованная Draft модель
    start = time.time()
    _, quant_stats = speculative_sampling(prompt, 50, heavy_target, quantized_draft_model, tokenizer)
    quant_time = time.time() - start

    print(f"\nСравнение времени работы Draft моделей на одном промпте:")
    print(f"Оригинальная Draft модель: {orig_time:.2f} секунд")
    print(f"Квантованная Draft модель: {quant_time:.2f} секунд")
    print(f"Ускорение: {orig_time/quant_time:.2f}x")
    print(f"Acceptance Rate (оригинальная): {orig_stats['total_accepted']/orig_stats['total_drafted']:.2%}")
    print(f"Acceptance Rate (квантованная): {quant_stats['total_accepted']/quant_stats['total_drafted']:.2%}")

compare_models()


Сравнение времени работы Draft моделей на одном промпте:
Оригинальная Draft модель: 2.28 секунд
Квантованная Draft модель: 5.80 секунд
Ускорение: 0.39x
Acceptance Rate (оригинальная): 100.00%
Acceptance Rate (квантованная): 65.00%


### 2. LoRA Distillation
Представьте, что Draft модель плохо согласована с Target моделью. Попробуйте дообучить (Fine-Tune) Draft модель на наборе ответов Target модели, используя LoRA (Low-Rank Adaptation). Даже 500 примеров и 10-15 минут обучения на T4 могут повысить Acceptance Rate.

In [ ]:
!pip install -q peft transformers accelerate

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from peft import LoraConfig, get_peft_model, TaskType
import time
import random
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence
from transformers.modeling_outputs import CausalLMOutputWithPast
import os

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [ ]:
# такой же NanoQwen, но адаптирован для peft
class NanoQwen2(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size)
        self.layers = nn.ModuleList([NanoQwenBlock(config) for _ in range(config.num_hidden_layers)])
        self.norm = QwenRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

        self.cos, self.sin = precompute_freqs_cis(
            config.hidden_size // config.num_attention_heads,
            config.max_position_embeddings * 2
        )
        self.cos = self.cos.to(device)
        self.sin = self.sin.to(device)

    def prepare_inputs_for_generation(self, input_ids, **kwargs):
        return {
            "input_ids": input_ids,
        }

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        if input_ids is None:
            input_ids = kwargs.get('input_ids')

        if isinstance(input_ids, torch.Tensor) and input_ids.dim() == 2:
            pass
        elif isinstance(input_ids, dict):
            input_ids = input_ids.get('input_ids')
        elif 'input_ids' in kwargs:
            input_ids = kwargs['input_ids']

        x = self.embed_tokens(input_ids)
        seq_len = x.shape[1]

        if seq_len > self.cos.shape[0]:
            self.cos, self.sin = precompute_freqs_cis(
                self.config.hidden_size // self.config.num_attention_heads,
                seq_len * 2
            )
            self.cos = self.cos.to(device)
            self.sin = self.sin.to(device)

        cos_t = self.cos[:seq_len]
        sin_t = self.sin[:seq_len]

        for layer in self.layers:
            x = layer(x, cos_t, sin_t)

        x = self.norm(x)
        logits = self.lm_head(x)

        if labels is not None:
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            loss = loss_fct(
                shift_logits.view(-1, shift_logits.size(-1)),
                shift_labels.view(-1)
            )

            # вернём объект, совместимый с PEFT
            return CausalLMOutputWithPast(
                loss=loss,
                logits=logits,
                past_key_values=None
            )

        # для инференса возвращаем просто логиты
        return CausalLMOutputWithPast(logits=logits)

In [ ]:
# переопределяем модель, адаптировав к peft
draft_model2 = NanoQwen2(draft_config)
draft_model2, missing_keys, unexpected_keys = load_weights_manual(draft_model2, DRAFT_ID)
draft_model2.eval()

Скачивание весов...


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Загрузка файла safetensors...
Связывание весов: lm_head.weight <- embed_tokens.weight
Загрузка весов в модель...
✅ Веса успешно загружены!


NanoQwen2(
  (embed_tokens): Embedding(151936, 896)
  (layers): ModuleList(
    (0-23): 24 x NanoQwenBlock(
      (self_attn): QwenAttention(
        (q_proj): Linear(in_features=896, out_features=896, bias=True)
        (k_proj): Linear(in_features=896, out_features=128, bias=True)
        (v_proj): Linear(in_features=896, out_features=128, bias=True)
        (o_proj): Linear(in_features=896, out_features=896, bias=False)
      )
      (mlp): QwenMLP(
        (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
        (up_proj): Linear(in_features=896, out_features=4864, bias=False)
        (down_proj): Linear(in_features=4864, out_features=896, bias=False)
      )
      (input_layernorm): QwenRMSNorm()
      (post_attention_layernorm): QwenRMSNorm()
    )
  )
  (norm): QwenRMSNorm()
  (lm_head): Linear(in_features=896, out_features=151936, bias=False)
)

In [ ]:
class DistillationDataset(Dataset):
    def __init__(self, prompts, target_model, tokenizer, max_length=128):
        self.input_ids = []
        self.labels = []

        with torch.no_grad(), torch.amp.autocast(device_type="cuda", dtype=torch.float16):
            for prompt in tqdm(prompts, desc="Generating target responses"):
                messages = [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ]
                text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

                inputs = tokenizer(text, return_tensors="pt", max_length=max_length, truncation=True).to(device)
                # outputs = target_model.generate(
                #     **inputs,
                #     max_new_tokens=50,
                #     do_sample=True,
                #     temperature=0.7,
                #     top_p=0.9
                # )
                outputs = target_model.generate(
                    **inputs,
                    max_new_tokens=50,
                    do_sample=False, # жадная генерация
                    temperature=0.0,
                    pad_token_id=tokenizer.eos_token_id
                )

                full_sequence = outputs[0] # промпт + ответ

                if len(full_sequence) > 3:
                    self.input_ids.append(full_sequence[:-1])
                    self.labels.append(full_sequence[1:])

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "labels": self.labels[idx]
        }

In [ ]:
def create_distillation_dataset(num_samples=500):
    base_prompts = [
        "Write a Python function to calculate factorial.",
        "Explain how neural networks work.",
        "What is the capital of France?",
        "How to make pancakes?",
        "Explain quantum computing in simple terms.",
        "Write a poem about autumn.",
        "What are the benefits of exercise?",
        "How does a car engine work?",
        "Explain the water cycle.",
        "Write a JavaScript function to reverse a string."
    ]

    prompts = []
    for _ in range(num_samples):
        base = random.choice(base_prompts)
        variation = random.choice(["", "Be concise.", "Be detailed.", "Explain step by step."])
        prompts.append(f"{base} {variation}".strip())

    return DistillationDataset(prompts[:num_samples], target_model, tokenizer)

In [ ]:
# конфигурация LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8, # Rank матрицы
    lora_alpha=32, # Масштабирующий коэффициент
    lora_dropout=0.05, # Dropout
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], # Слои для адаптации
    bias="none"
)

In [ ]:
# применяем LoRA к Draft модели
lora_model = get_peft_model(draft_model2, lora_config)
lora_model.print_trainable_parameters()

trainable params: 4,399,104 || all params: 634,566,528 || trainable%: 0.6932


In [ ]:
def train_lora_model(lora_model, dataset, target_model, tokenizer,
                    epochs=3, batch_size=4, learning_rate=1e-4):
    lora_model = lora_model.to(device)
    lora_model.train()
    optimizer = torch.optim.AdamW(lora_model.parameters(), lr=learning_rate)

    val_prompts = [
        "Write a Python function to calculate Fibonacci numbers.",
        "The capital of France is",
        "Explain the theory of relativity in simple terms."
    ]

    def collate_fn(batch):
        input_ids = [item["input_ids"] for item in batch]
        labels = [item["labels"] for item in batch]

        max_len = max(len(ids) for ids in input_ids)

        padded_input_ids = pad_sequence(
            input_ids,
            batch_first=True,
            padding_value=tokenizer.pad_token_id
        )

        padded_labels = pad_sequence(
            labels,
            batch_first=True,
            padding_value=-100
        )

        return {
            "input_ids": padded_input_ids,
            "labels": padded_labels
        }

    print(f"Начало обучения для спекулятивного декодирования на {len(dataset)} примерах...")

    for epoch in range(epochs):
        total_loss = 0
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
        progress_bar = tqdm(dataloader, desc=f"Эпоха {epoch+1}/{epochs}")

        for batch in progress_bar:
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            draft_outputs = lora_model(input_ids=input_ids, labels=labels)
            draft_logits = draft_outputs.logits

            with torch.no_grad():
                target_outputs = target_model(input_ids=input_ids)
                target_logits = target_outputs.logits

            # обрезка до одинаковой длины
            min_seq_len = min(draft_logits.shape[1], target_logits.shape[1], labels.shape[1] + 1)

            # обрезка логитов для KL дивергенции
            draft_logits_kl = draft_logits[:, :min_seq_len-1, :]
            target_logits_kl = target_logits[:, :min_seq_len-1, :]

            # KL дивергенция
            draft_log_probs = F.log_softmax(draft_logits_kl, dim=-1)
            target_probs = F.softmax(target_logits_kl, dim=-1)
            kl_loss = F.kl_div(draft_log_probs, target_probs, reduction='batchmean')

            # CE loss для следующего токена
            shift_logits = draft_logits[:, :min_seq_len-1, :].reshape(-1, draft_logits.size(-1))
            shift_labels = labels[:, :min_seq_len-1].reshape(-1)
            ce_loss = F.cross_entropy(
                shift_logits,
                shift_labels,
                ignore_index=-100
            )

            # Комбинированный loss
            loss = kl_loss + 0.5 * ce_loss

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix({"loss": f"{loss.item():.4f}", "kl_loss": f"{kl_loss.item():.4f}", "ce_loss": f"{ce_loss.item():.4f}"})

        # считаем Acceptance Rate после каждой эпохи
        acceptance_rate = evaluate_acceptance_rate(target_model, lora_model, val_prompts)
        avg_loss = total_loss / len(progress_bar)
        print(f"Эпоха {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Acceptance Rate: {acceptance_rate:.2%}")

        if epoch > 0 and acceptance_rate < 0.5:
            print("Acceptance Rate слишком низкий. Останавливаем обучение.")
            break

    lora_model.eval()
    print("Обучение для спекулятивного декодирования завершено!")
    return lora_model

In [ ]:
def speculative_sampling(prefix_text, max_new_tokens, target_model, draft_model, tokenizer, K=5):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prefix_text}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)

    generated_ids = input_ids.clone()
    finished_len = input_ids.shape[1] + max_new_tokens

    stats = {"target_calls": 0, "total_accepted": 0, "total_drafted": 0}

    with torch.no_grad(), torch.amp.autocast(device_type="cuda", dtype=torch.float16):
        while generated_ids.shape[1] < finished_len:
            prefix_len = generated_ids.shape[1]

            # DRAFT
            draft_ids = generated_ids
            for _ in range(K):
                outputs = draft_model(input_ids=draft_ids)
                if isinstance(outputs, dict):
                    logits = outputs["logits"]
                else:
                    logits = outputs

                next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
                draft_ids = torch.cat([draft_ids, next_token], dim=1)
                if next_token.item() == tokenizer.eos_token_id:
                    break

            drafted_tokens = draft_ids[0, prefix_len:]
            if not len(drafted_tokens): break
            stats["total_drafted"] += len(drafted_tokens)

            outputs = target_model(input_ids=draft_ids)
            if isinstance(outputs, dict) and "logits" in outputs:
                target_logits = outputs["logits"]
            else:
                target_logits = outputs.logits if hasattr(outputs, "logits") else outputs

            stats["target_calls"] += 1

            relevant_logits = target_logits[0, prefix_len-1 : -1]
            target_preds = torch.argmax(relevant_logits, dim=-1)

            n_accepted = 0
            for i in range(len(drafted_tokens)):
                if i >= len(target_preds):
                    break
                if drafted_tokens[i] == target_preds[i]:
                    n_accepted += 1
                else:
                    break

            stats["total_accepted"] += n_accepted

            accepted_ids = drafted_tokens[:n_accepted]
            generated_ids = torch.cat([generated_ids, accepted_ids.unsqueeze(0)], dim=1)

            if generated_ids.shape[1] >= finished_len:
                break

            if n_accepted < len(target_preds):
                correct_token = target_preds[n_accepted].view(1, 1)
            else:
                if isinstance(outputs, dict) and "logits" in outputs:
                    last_logits = outputs["logits"][0, -1, :]
                else:
                    last_logits = outputs.logits[0, -1, :] if hasattr(outputs, "logits") else outputs[0, -1, :]
                correct_token = torch.argmax(last_logits).view(1, 1)

            generated_ids = torch.cat([generated_ids, correct_token], dim=1)

            if correct_token.item() == tokenizer.eos_token_id:
                break

    return tokenizer.decode(generated_ids[0], skip_special_tokens=True), stats

In [ ]:
# метрика Acceptance Rate
def evaluate_acceptance_rate(target_model, draft_model, prompts, K=5, device_type="cuda"):
    stats = {"total_accepted": 0, "total_drafted": 0}

    with torch.no_grad(), torch.amp.autocast(device_type=device_type, dtype=torch.float16):
        for prompt in prompts:
            _, prompt_stats = speculative_sampling(prompt, 50, target_model, draft_model, tokenizer, K=K)
            stats["total_accepted"] += prompt_stats["total_accepted"]
            stats["total_drafted"] += prompt_stats["total_drafted"]

    acceptance_rate = stats["total_accepted"] / max(stats["total_drafted"], 1)
    return acceptance_rate

In [ ]:
dataset = create_distillation_dataset(num_samples=500)

Generating target responses: 100%|██████████| 500/500 [16:19<00:00,  1.96s/it]


In [ ]:
# отобразим dataset
for i in range(30):
    example = dataset[i]
    # print(example)
    input_ids = example["input_ids"]
    labels = example["labels"]

    # Декодируем в читаемый текст
    input_text = tokenizer.decode(input_ids, skip_special_tokens=False)
    label_text = tokenizer.decode(labels, skip_special_tokens=False)

    print(f"Пример #{i+1}:")
    print(f"Длина последовательности: {len(input_ids)} токенов")
    print("-" * 80)
    print(f"➡️ Вход (input_ids → текст):\n    {input_text}")
    print("-" * 80)
    print(f"🎯 Цель (labels → текст):\n    {label_text}")
    print("=" * 80)

Пример #1:
Длина последовательности: 78 токенов
--------------------------------------------------------------------------------
➡️ Вход (input_ids → текст):
    <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
How does a car engine work? Be concise.<|im_end|>
<|im_start|>assistant
A car engine converts fuel into mechanical energy to power the vehicle. It consists of an internal combustion system where air and fuel mix in cylinders, igniting with spark plugs to create explosions that turn pistons, which drive the crankshaft connected to the
--------------------------------------------------------------------------------
🎯 Цель (labels → текст):
    system
You are a helpful assistant.<|im_end|>
<|im_start|>user
How does a car engine work? Be concise.<|im_end|>
<|im_start|>assistant
A car engine converts fuel into mechanical energy to power the vehicle. It consists of an internal combustion system where air and fuel mix in cylinders, igniting with spark plugs to

In [ ]:
# оценка Acceptance Rate до обучения
prompts_eval = [
    "Write a Python function to calculate Fibonacci numbers.",
    "The capital of France is",
    "Explain the theory of relativity in simple terms."
]
acceptance_before = evaluate_acceptance_rate(target_model, draft_model2, prompts_eval)
print(f"Acceptance Rate до обучения: {acceptance_before:.2%}")

Acceptance Rate до обучения: 65.24%


In [ ]:
# обучение LoRA адаптеров
lora_model = train_lora_model(lora_model, dataset, target_model, tokenizer, epochs=3, batch_size=4)

Начало обучения для спекулятивного декодирования на 500 примерах...


Эпоха 1/3: 100%|██████████| 125/125 [00:33<00:00,  3.75it/s, loss=8.0547, kl_loss=7.3125, ce_loss=1.4824]


Эпоха 1/3 | Loss: 15.3754 | Acceptance Rate: 66.48%


Эпоха 2/3: 100%|██████████| 125/125 [00:30<00:00,  4.04it/s, loss=4.6758, kl_loss=4.0508, ce_loss=1.2480]


Эпоха 2/3 | Loss: 6.7361 | Acceptance Rate: 64.17%


Эпоха 3/3: 100%|██████████| 125/125 [00:31<00:00,  3.94it/s, loss=9.1797, kl_loss=8.5156, ce_loss=1.3223]


Эпоха 3/3 | Loss: 5.3777 | Acceptance Rate: 67.03%
Обучение для спекулятивного декодирования завершено!


In [ ]:
# оценка Acceptance Rate после обучения
acceptance_after = evaluate_acceptance_rate(target_model, lora_model, prompts_eval)
print(f"Acceptance Rate после обучения: {acceptance_after:.2%}")
print(f"Улучшение: {acceptance_after - acceptance_before:.2%}")

Acceptance Rate после обучения: 67.03%
Улучшение: 1.79%


In [ ]:
# сохраняем обученную LoRA модель
lora_model.save_pretrained("lora_nanoqwen")

### 3. Layer Pruning Distillation

Возьмите Target модель (1.5B) и создайте из нее Draft модель, просто удалив половину слоев (например, каждый второй). Получится "покалеченная" модель ~0.8B. Затем проведите Knowledge Distillation (обучите её восстанавливать логиты оригинальной модели) и используйте результат как Draft модель.


In [ ]:
# torch.cuda.empty_cache()
# gc.collect()

In [1]:
!pip install -q peft transformers accelerate

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import time
import random
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence
import copy

In [ ]:
# torch.backends.cuda.matmul.allow_tf32 = True
# torch.backends.cudnn.allow_tf32 = True

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [6]:
device

'cuda:0'

In [4]:
TARGET_ID = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(TARGET_ID)

target_model = AutoModelForCausalLM.from_pretrained(
    TARGET_ID,
    dtype="auto",
).to(device)

target_model.eval()
target_model.requires_grad_(False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotar

In [10]:
# оригинальное количество слоёв
target_model.config.num_hidden_layers

28

In [5]:
def create_pruned_model(teacher):
    """
    Создаёт student модель, удаляя каждый второй слой teacher.
    """

    config = copy.deepcopy(teacher.config)
    config.num_hidden_layers //= 2
    config.use_cache = False

    student = AutoModelForCausalLM.from_config(config)

    student.model.embed_tokens.load_state_dict(
        copy.deepcopy(teacher.model.embed_tokens.state_dict())
    )

    new_layers = []
    for layer in teacher.model.layers[::2]:
        new_layers.append(copy.deepcopy(layer))

    student.model.layers = torch.nn.ModuleList(new_layers)

    if hasattr(teacher.model, "norm"):
        student.model.norm.load_state_dict(
            copy.deepcopy(teacher.model.norm.state_dict())
        )

    student.lm_head.load_state_dict(
        copy.deepcopy(teacher.lm_head.state_dict())
    )

    return student

In [6]:
from torch.utils.data import Dataset

class DistillationDataset(Dataset):
    def __init__(self, prompts, tokenizer, max_length=128):
        self.input_ids = []
        self.attention_masks = []

        for prompt in prompts:
            messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt},
            ]

            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            enc = tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                max_length=max_length,
                padding="max_length"  # ВАЖНО
            )

            self.input_ids.append(enc.input_ids.squeeze(0))
            self.attention_masks.append(enc.attention_mask.squeeze(0))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_masks[idx],
        }

In [7]:
def make_prompts(n=500):
    base = [
        "Explain how neural networks work.",
        "Write a Python function for factorial.",
        "What is the capital of France?",
        "Explain quantum computing simply.",
        "How does a car engine work?",
        "Describe the water cycle.",
        "What is photosynthesis?",
        "Explain blockchain technology.",
        "How does GPS work?",
        "What is machine learning?",
    ]
    mods = [
        "", "Be concise.", "Be detailed.", "Explain step by step.",
        "Use analogies.", "Provide examples.", "For beginners.",
        "With code examples.", "Compare with alternatives.",
        "In 3 bullet points.", "As a story.", "For experts."
    ]
    return [random.choice(base) + " " + random.choice(mods) for _ in range(n)]

In [ ]:
device

'cuda:0'

In [9]:
def distillation_loss(
    student_logits,
    teacher_logits,
    attention_mask,
    temperature=2.0,
    alpha=0.7
):
    min_seq_len = min(
        student_logits.size(1),
        teacher_logits.size(1),
        attention_mask.size(1) - 1
    )

    student_logits = student_logits[:, :min_seq_len, :]
    teacher_logits = teacher_logits[:, :min_seq_len, :]
    student_mask = attention_mask[:, 1:min_seq_len+1].float()

    student_log_probs = F.log_softmax(student_logits.float() / temperature, dim=-1)
    teacher_probs = F.softmax(teacher_logits.float() / temperature, dim=-1)

    kl = F.kl_div(
        student_log_probs,
        teacher_probs,
        reduction="none",
        log_target=False
    ).sum(-1)

    kl_loss = (kl * student_mask).sum() / (student_mask.sum() + 1e-8)

    with torch.no_grad():
        hard_targets = teacher_logits.float().argmax(dim=-1)

    ce_loss = F.cross_entropy(
        student_logits.float().view(-1, student_logits.size(-1)),
        hard_targets.view(-1),
        reduction="none",
        ignore_index=-100
    ).view_as(student_mask)

    ce_loss = (ce_loss * student_mask).sum() / (student_mask.sum() + 1e-8)

    total_loss = alpha * kl_loss + (1 - alpha) * ce_loss

    if torch.isnan(total_loss) or torch.isinf(total_loss):
        print(f"Warning: NaN/Inf в total_loss. kl_loss={kl_loss.item():.4f}, ce_loss={ce_loss.item():.4f}")
        print(f"student_logits min/max: {student_logits.min().item():.4f}/{student_logits.max().item():.4f}")
        print(f"teacher_logits min/max: {teacher_logits.min().item():.4f}/{teacher_logits.max().item():.4f}")

        return torch.tensor(0.0, device=total_loss.device, requires_grad=True)

    return total_loss

In [10]:
def distill(
    student,
    teacher,
    dataset,
    epochs=3,
    lr=1e-5,
    batch_size=4,
    temperature=2.0,
    gradient_clip=0.5
):
    teacher.eval()
    teacher.requires_grad_(False)
    teacher = teacher.to(device)

    student = student.to(device)
    student.train()

    if hasattr(student, 'gradient_checkpointing_disable'):
        student.gradient_checkpointing_disable()

    optimizer = torch.optim.AdamW(student.parameters(), lr=lr, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(dataset) * epochs)

    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

    for epoch in range(epochs):
        total_loss = 0.0
        total_kl = 0.0
        total_ce = 0.0
        num_batches = 0
        num_skipped = 0

        bar = tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}")
        for batch in bar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            with torch.no_grad(), torch.amp.autocast('cuda', dtype=torch.float16):
                teacher_outputs = teacher(input_ids, attention_mask=attention_mask)
                teacher_logits = teacher_outputs.logits[:, :-1, :]

            with torch.amp.autocast('cuda', dtype=torch.float16):
                student_outputs = student(input_ids, attention_mask=attention_mask)
                student_logits = student_outputs.logits[:, :-1, :]

            with torch.amp.autocast('cuda', enabled=False):
                student_mask = attention_mask[:, 1:].float()

                min_seq_len = min(
                    student_logits.size(1),
                    teacher_logits.size(1),
                    student_mask.size(1)
                )
                student_logits = student_logits[:, :min_seq_len, :]
                teacher_logits = teacher_logits[:, :min_seq_len, :]
                student_mask = student_mask[:, :min_seq_len]

                loss = distillation_loss(
                    student_logits,
                    teacher_logits,
                    student_mask,
                    temperature,
                    alpha=0.7
                )

            if torch.isnan(loss) or torch.isinf(loss) or loss.item() > 100.0:
                print(f"Warning: Пропуск батча: loss={loss.item():.4f}")
                num_skipped += 1
                optimizer.zero_grad()
                continue

            optimizer.zero_grad()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(student.parameters(), gradient_clip)

            has_nan_grad = False
            for name, param in student.named_parameters():
                if param.grad is not None:
                    if torch.isnan(param.grad).any() or torch.isinf(param.grad).any():
                        print(f"Warning: NaN/Inf gradient in {name}")
                        has_nan_grad = True
                        break

            if not has_nan_grad:
                optimizer.step()
                scheduler.step()

            total_loss += loss.item()
            num_batches += 1

            bar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'lr': f"{scheduler.get_last_lr()[0]:.2e}",
                'skipped': num_skipped
            })

        avg_loss = total_loss / max(num_batches, 1)
        print(f"Epoch {epoch+1}/{epochs} | Avg loss: {avg_loss:.4f} | Skipped batches: {num_skipped}/{len(loader)}")

        with torch.no_grad():
            student.eval()
            kl_val = evaluate_kl(student, teacher, val_dataset, temperature=2.0)
            print(f"Validation KL after epoch {epoch+1}: {kl_val:.4f}")
            student.train()

    torch.cuda.empty_cache()
    student.eval()
    return student

In [11]:
prompts = make_prompts(500)

In [17]:
prompts

['What is the capital of France? Use analogies.',
 'Describe the water cycle. In 3 bullet points.',
 'Write a Python function for factorial. For beginners.',
 'Write a Python function for factorial. Be detailed.',
 'Describe the water cycle. Provide examples.',
 'How does a car engine work? Use analogies.',
 'What is the capital of France? ',
 'What is the capital of France? As a story.',
 'What is the capital of France? Provide examples.',
 'Describe the water cycle. For beginners.',
 'Write a Python function for factorial. With code examples.',
 'Describe the water cycle. For beginners.',
 'What is photosynthesis? In 3 bullet points.',
 'Write a Python function for factorial. ',
 'What is photosynthesis? Explain step by step.',
 'Explain blockchain technology. With code examples.',
 'What is photosynthesis? Compare with alternatives.',
 'Describe the water cycle. In 3 bullet points.',
 'What is photosynthesis? With code examples.',
 'What is machine learning? As a story.',
 'How does

In [12]:
dataset = DistillationDataset(prompts, tokenizer, max_length=128)

In [13]:
val_prompts = make_prompts(50)
val_dataset = DistillationDataset(
    val_prompts,
    tokenizer,
    max_length=128
)

In [14]:
@torch.no_grad()
def evaluate_kl(student, teacher, dataset, temperature=2.0):
    student.eval()
    teacher.eval()

    loader = DataLoader(dataset, batch_size=1)
    total_kl = 0.0
    n_tokens = 0

    for batch in loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        with torch.amp.autocast("cuda", dtype=torch.float16):
            t_logits = teacher(input_ids).logits[:, :-1]
            s_logits = student(input_ids).logits[:, :-1]

        t_probs = torch.softmax(t_logits.float() / temperature, dim=-1)
        s_log_probs = torch.log_softmax(s_logits.float() / temperature, dim=-1)

        kl = torch.nn.functional.kl_div(
            s_log_probs,
            t_probs,
            reduction="none"
        ).sum(-1)

        mask = attention_mask[:, 1:]
        total_kl += (kl * mask).sum().item()
        n_tokens += mask.sum().item()

    return total_kl / max(n_tokens, 1)

In [15]:
draft_model = create_pruned_model(target_model)

In [16]:
draft_model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-13): 14 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotar

In [23]:
print(f"Количество слоев в пруненной модели: {draft_model.config.num_hidden_layers}")

Количество слоев в пруненной модели: 14


In [24]:
target_model.device

device(type='cuda', index=0)

In [25]:
draft_model.device

device(type='cpu')

In [26]:
device

'cuda:0'

In [17]:
kl_before = evaluate_kl(draft_model.to(device), target_model, val_dataset)
print(f"KL before distillation: {kl_before:.4f}")

KL before distillation: 3.4677


In [18]:
draft_model = distill(
    student=draft_model,
    teacher=target_model,
    dataset=dataset,
    epochs=3
)

Epoch 1/3: 100%|██████████| 125/125 [00:53<00:00,  2.35it/s, loss=4.7343, lr=9.83e-06, skipped=0]


Epoch 1/3 | Avg loss: 5.2877 | Skipped batches: 0/125
Validation KL after epoch 1: 3.0343


Epoch 2/3: 100%|██████████| 125/125 [00:50<00:00,  2.46it/s, loss=4.1153, lr=9.33e-06, skipped=0]


Epoch 2/3 | Avg loss: 4.4024 | Skipped batches: 0/125
Validation KL after epoch 2: 2.8645


Epoch 3/3: 100%|██████████| 125/125 [00:51<00:00,  2.44it/s, loss=3.9265, lr=8.54e-06, skipped=0]


Epoch 3/3 | Avg loss: 4.0041 | Skipped batches: 0/125
Validation KL after epoch 3: 2.7549


In [19]:
torch.save(draft_model.state_dict(), "draft_qwen_kd.pt")

In [ ]:
target_model.device

device(type='cuda', index=0)

In [ ]:
draft_model.device

device(type='cuda', index=0)

In [20]:
kl_after = evaluate_kl(draft_model, target_model, val_dataset)
print(f"KL after distillation:  {kl_after:.4f}")

KL after distillation:  2.7549


In [21]:
prompt = "Explain how neural networks work."
inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    print("Teacher:")
    print(tokenizer.decode(
        target_model.generate(**inputs, max_new_tokens=80)[0]
    ))

    print("\nDraft:")
    print(tokenizer.decode(
        draft_model.generate(**inputs, max_new_tokens=80)[0]
    ))

Teacher:


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Explain how neural networks work. Neural networks are a type of machine learning algorithm that is inspired by the structure and function of the human brain. They consist of interconnected nodes, or neurons, which receive input data and pass it on to other neurons in subsequent layers.

At each layer of a neural network, there are many neurons connected to the previous layer's neurons. The output of each neuron is calculated using a mathematical function called an activation

Draft:
Explain how neural networks work. Howaz X Flash proscenespeed told旁part yetind howskibo
另外, ek,Part canah off,深入何:Book造in anel探讨acesubicremaininglo้ง,。 h far 公 Wholesaleinnbatt,off in order up
ways will icesure sErain and,<|endoftext|>erty in pro(pystrled Waitersenha role pass case form
